# Code Explainer

In this we are going to build a code explainer that will create a docstring of a given function!

## Library Imports

In [1]:
import os
import inspect
import openai

In [2]:
# Set the OpenAI Key
openai.api_key = os.getenv('OPENAI_API_KEY')

We are going to set some rules 
- The language is Python
- It has to be a function for now

## Prompt Design

In [3]:
''' We define two functions '''
def docstring_prompt(code):
    prompt = f"{code}\n # A high quality python docstring of the above python function : \n\"\"\""
    return prompt

In [6]:
'''We can use inspect to get the source of the function'''
def hello(name):
    print (f"Hello {name}")

In [7]:
print(inspect.getsource(hello)) 

def hello(name):
    print (f"Hello {name}")


In [10]:
'''Now we make a OpenAI call to generate the docstring of the simple hello function'''

response = openai.Completion.create(model='text-davinci-003',
                                    prompt = docstring_prompt(inspect.getsource(hello)),
                                    temperature = 0,
                                    max_tokens = 64,
                                    top_p = 1.0,
                                    frequency_penalty = 0.0,
                                    presence_penalty = 0.0,
                                    stop = ["\"\"\""]
                                    )

In [11]:
''' Check the docstring response '''
print(response['choices'][0]['text'])


This function prints a greeting to the given name.

Parameters:
    name (str): The name of the person to greet.

Returns:
    None


In [12]:
'''Now we will combine the docstring and the function and output everything together'''
def merge_docstring_function(original_function, docstring):
    function_string = inspect.getsource(original_function)
    split = function_string.split("\n")
    first_part, second_part = split[0], split[1:]
    
    '''Note the spaces before the trim'''
    merged_function = first_part + "\n" + '    """' + docstring + '    """' + "\n" + "\n".join(second_part)
    return merged_function

In [13]:
final_code = merge_docstring_function(hello, response['choices'][0]['text'])

In [14]:
print(final_code)

def hello(name):
    """
This function prints a greeting to the given name.

Parameters:
    name (str): The name of the person to greet.

Returns:
    None
    """
    print (f"Hello {name}")


## Get the Docstring of a more complicated function

In [15]:
def test_function(test, num_questions):
    student_view = {1 : ""}
    question_number = 1
    for line in test.split("\n"):
        if not line.startswith("Correct Answer:"):
            student_view[question_number] += line+"\n"
        else:

            if question_number < num_questions:
                question_number+=1
                student_view[question_number] = ""
    return student_view

In [16]:
response = openai.Completion.create(model = 'text-davinci-003',
                                    prompt = docstring_prompt(inspect.getsource(test_function)),
                                    temperature = 0,
                                    max_tokens = 256,
                                    top_p = 1.0,
                                    frequency_penalty=0.0,
                                    presence_penalty=0.0,
                                    stop = ["\"\"\""]  # Corresponds to """, the end of the docstring
                                    )

In [17]:
final_code = merge_docstring_function(test_function, response['choices'][0]['text'])
print(final_code)

def test_function(test, num_questions):
    """
This function takes a test string and the number of questions as input and returns a dictionary with the questions as keys and the corresponding question text as values.

Parameters:
test (str): The test string containing the questions and answers.
num_questions (int): The number of questions in the test.

Returns:
student_view (dict): A dictionary with the questions as keys and the corresponding question text as values.

    """
    student_view = {1 : ""}
    question_number = 1
    for line in test.split("\n"):
        if not line.startswith("Correct Answer:"):
            student_view[question_number] += line+"\n"
        else:

            if question_number < num_questions:
                question_number+=1
                student_view[question_number] = ""
    return student_view

